# My KNN implementation

In [1]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np
import random

In [2]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [3]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [4]:
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [5]:
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        column_values = [row[i] for row in dataset]
        value_min = min(column_values)
        value_max = max(column_values)
        minmax.append([value_min, value_max])
    return minmax

In [6]:
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [7]:
def knn(train_data, test_data, class_index, k, dist_func):
    distances = []

    for train_point, label in train_data:
        dist = dist_func(train_point, test_data)
        distances.append((dist, label))

    sorted_distances = sorted(distances, key=lambda x: x[0])
    k_nearest_neighbors = sorted_distances[:k]

    votes_of_each_class= dict()
    for dist,label in k_nearest_neighbors:
        if label in votes_of_each_class:
            votes_of_each_class[label]+=1
        else:
            votes_of_each_class[label] = 1
            
    return max(votes_of_each_class, key=votes_of_each_class.get)

In [85]:
def k_fold_cross_validation(data, num_of_folds, class_index, dist_func,k):
    n = len(data)
    fold_size = n //num_of_folds
    accuracies=[]
    random.Random(42).shuffle(data)
    
    
    for i in range(num_of_folds):
        test_data = data[i * fold_size:(i + 1)* fold_size]
        train_data = data[:i *fold_size] + data[(i+1) * fold_size:]
        
        correct_identify=0
        for test_point, true_label in test_data:
            predicted_label = knn(train_data,test_point,class_index, k, dist_func)
            if predicted_label == true_label:
                correct_identify += 1

        accuracy = correct_identify /len(test_data)
        accuracies.append(accuracy)

    return accuracies

In [9]:
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

In [10]:
def cosine_distance(point1, point2):
    dot_product= np.dot(point1, point2)
    norm1 = np.linalg.norm(point1)
    norm2 = np.linalg.norm(point2)
    cosine_similarity= dot_product / (norm1 * norm2)
    cosine_distance = 1 - cosine_similarity
    return cosine_distance

In [11]:
def hamming_distance(point1, point2):
    distance = sum(bit1 != bit2 for bit1, bit2 in zip(point1, point2))
    return distance

In [12]:
def manhattan_distance(point1, point2):
    distance = sum(abs(x - y) for x, y in zip(point1, point2))
    return distance

In [13]:
def minkowski_distance(point1, point2, p=2):
    distance = sum(abs(x - y) ** p for x, y in zip(point1, point2)) ** (1/p)
    return distance

In [86]:
def run_knn_k_cross(filename,class_index,list_of_encoding_index,dist_func,k):
    dataset = load_csv(filename)
    column_lookup = dict()
    for i in range(len(dataset[0])):
        if i != class_index and i not in list_of_encoding_index:
            str_column_to_float(dataset, i)
        elif i in list_of_encoding_index:
            column_lookup[i] = str_column_to_int(dataset, i)

    column_lookup[class_index]=str_column_to_int(dataset, class_index)

    n_folds = 10
    new_dataset= []
    normalize_dataset(dataset,dataset_minmax(dataset))
    for i in range(len(dataset)):
        label = dataset[i][class_index]
        dataset[i].pop(class_index)
        new_dataset.append([tuple(dataset[i]),label])
    
    scores=k_fold_cross_validation(new_dataset, n_folds, class_index, dist_func, k)
    print('Scores:', scores)
    print("Mean Accuracy: {:.3f}".format(sum(scores)/float(len(scores))))

## Checking accuracy with euclidean distance

In [141]:
run_knn_k_cross("breast-cancer.data",0,[1,2,3,4,5,7,8,9],euclidean_distance,7)

Scores: [0.6071428571428571, 0.7857142857142857, 0.6785714285714286, 0.7857142857142857, 0.6785714285714286, 0.7142857142857143, 0.8214285714285714, 0.8214285714285714, 0.75, 0.8214285714285714]
Mean Accuracy: 0.746


In [99]:
run_knn_k_cross("hayes-roth.data",5,[],euclidean_distance,2)

Scores: [0.6923076923076923, 0.7692307692307693, 0.6923076923076923, 0.6153846153846154, 0.7692307692307693, 0.6923076923076923, 0.6923076923076923, 0.7692307692307693, 0.7692307692307693, 0.6923076923076923]
Mean Accuracy: 0.715


In [154]:
run_knn_k_cross("car.data",6,[0,1,2,3,4,5],euclidean_distance,9)

Scores: [0.7093023255813954, 0.8081395348837209, 0.7325581395348837, 0.8313953488372093, 0.7790697674418605, 0.7965116279069767, 0.7616279069767442, 0.7209302325581395, 0.7441860465116279, 0.7558139534883721]
Mean Accuracy: 0.764


## Checking accuracy with hamming distance

In [123]:
run_knn_k_cross("breast-cancer.data",0,[1,2,3,4,5,7,8,9],hamming_distance,7)

Scores: [0.6071428571428571, 0.8214285714285714, 0.6785714285714286, 0.75, 0.6428571428571429, 0.75, 0.75, 0.7857142857142857, 0.75, 0.8214285714285714]
Mean Accuracy: 0.736


In [119]:
run_knn_k_cross("hayes-roth.data",5,[],hamming_distance,2)

Scores: [0.6923076923076923, 0.6923076923076923, 0.6153846153846154, 0.6923076923076923, 0.6923076923076923, 0.7692307692307693, 0.9230769230769231, 0.8461538461538461, 0.46153846153846156, 0.6153846153846154]
Mean Accuracy: 0.700


In [114]:
run_knn_k_cross("car.data",6,[0,1,2,3,4,5],hamming_distance,9)

Scores: [0.877906976744186, 0.936046511627907, 0.8895348837209303, 0.9418604651162791, 0.9186046511627907, 0.9418604651162791, 0.9302325581395349, 0.9244186046511628, 0.8953488372093024, 0.9244186046511628]
Mean Accuracy: 0.918


## Checking accuracy with cosine distance

In [129]:
run_knn_k_cross("breast-cancer.data",0,[1,2,3,4,5,7,8,9],cosine_distance,7)

Scores: [0.6071428571428571, 0.75, 0.7142857142857143, 0.7857142857142857, 0.7142857142857143, 0.7857142857142857, 0.8214285714285714, 0.7857142857142857, 0.7857142857142857, 0.8214285714285714]
Mean Accuracy: 0.757


In [143]:
run_knn_k_cross("hayes-roth.data",5,[],cosine_distance,2)

Scores: [0.6153846153846154, 0.6153846153846154, 0.5384615384615384, 0.5384615384615384, 0.8461538461538461, 0.7692307692307693, 0.6153846153846154, 0.6153846153846154, 0.6923076923076923, 0.6923076923076923]
Mean Accuracy: 0.654


In [153]:
run_knn_k_cross("car.data",6,[0,1,2,3,4,5],cosine_distance,9)

/var/folders/nw/s4h_7xbn2_d8cf322fxsyjbr0000gp/T/ipykernel_75666/2619794587.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity= dot_product / (norm1 * norm2)


Scores: [0.7151162790697675, 0.7732558139534884, 0.7093023255813954, 0.7965116279069767, 0.7383720930232558, 0.7906976744186046, 0.7209302325581395, 0.7034883720930233, 0.7267441860465116, 0.7093023255813954]
Mean Accuracy: 0.738


## Checking accuracy with manhattan distance

In [138]:
run_knn_k_cross("breast-cancer.data",0,[1,2,3,4,5,7,8,9],manhattan_distance,7)

Scores: [0.6071428571428571, 0.7857142857142857, 0.6785714285714286, 0.75, 0.6428571428571429, 0.7142857142857143, 0.7857142857142857, 0.8214285714285714, 0.75, 0.7857142857142857]
Mean Accuracy: 0.732


In [142]:
run_knn_k_cross("hayes-roth.data",5,[],manhattan_distance,2)

Scores: [0.6153846153846154, 0.7692307692307693, 0.6923076923076923, 0.6153846153846154, 0.7692307692307693, 0.6923076923076923, 0.6923076923076923, 0.7692307692307693, 0.7692307692307693, 0.6153846153846154]
Mean Accuracy: 0.700


In [152]:
run_knn_k_cross("car.data",6,[0,1,2,3,4,5],manhattan_distance,9)

Scores: [0.8372093023255814, 0.9127906976744186, 0.8546511627906976, 0.9069767441860465, 0.8895348837209303, 0.9127906976744186, 0.8895348837209303, 0.8255813953488372, 0.8546511627906976, 0.8313953488372093]
Mean Accuracy: 0.872


## Checking accuracy with minkowski distance

In [149]:
run_knn_k_cross("breast-cancer.data",0,[1,2,3,4,5,7,8,9],minkowski_distance,7)

Scores: [0.6071428571428571, 0.7857142857142857, 0.6785714285714286, 0.7857142857142857, 0.6785714285714286, 0.7142857142857143, 0.8214285714285714, 0.8214285714285714, 0.75, 0.8214285714285714]
Mean Accuracy: 0.746


In [150]:
run_knn_k_cross("hayes-roth.data",5,[],minkowski_distance,2)

Scores: [0.6923076923076923, 0.7692307692307693, 0.6923076923076923, 0.6153846153846154, 0.7692307692307693, 0.6923076923076923, 0.6923076923076923, 0.7692307692307693, 0.7692307692307693, 0.6923076923076923]
Mean Accuracy: 0.715


In [151]:
run_knn_k_cross("car.data",6,[0,1,2,3,4,5],minkowski_distance,9)

Scores: [0.7093023255813954, 0.8081395348837209, 0.7325581395348837, 0.8313953488372093, 0.7790697674418605, 0.7965116279069767, 0.7616279069767442, 0.7209302325581395, 0.7441860465116279, 0.7558139534883721]
Mean Accuracy: 0.764


# Sklearn KNN

In [81]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from csv import reader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score


def sklearn_knn(filename, class_index, list_of_encoding_index, k,num_of_folds=10):
    #can specify no. of neighbors(k) as argument
    dataset = load_csv(filename)
    column_lookup = dict()
    for i in range(len(dataset[0])):
        if i != class_index and i not in list_of_encoding_index:
            str_column_to_float(dataset, i)
        elif i in list_of_encoding_index:
            column_lookup[i] = str_column_to_int(dataset, i)
            
    column_lookup[class_index]=str_column_to_int(dataset, class_index)
    new_dataset=[]

    for i in range(len(dataset)):
        label = dataset[i][class_index]
        dataset[i].pop(class_index)
        new_dataset.append([tuple(dataset[i]),label])
    dataset = new_dataset
    n = len(dataset)
    fold_size = n //num_of_folds
    random.Random(42).shuffle(dataset)
    scores=[]
    for i in range(num_of_folds):
        test_data = dataset[i * fold_size:(i + 1)* fold_size]
        train_data = dataset[:i *fold_size] + dataset[(i+1) * fold_size:]
    
        X_train = []
        X_test = []
        y_train = []
        y_test = []
    
        for train_point,label in train_data:
            X_train.append(train_point)
            y_train.append(label)

        for test_point,label in test_data:
            X_test.append(test_point)
            y_test.append(label)
            
        scaler = MinMaxScaler()
        knn = KNeighborsClassifier(n_neighbors=k)
    
        knn.fit(scaler.fit_transform(X_train), y_train)
        
        y_pred = knn.predict(scaler.fit_transform(X_test))
        
        accuracy = accuracy_score(y_test, y_pred)
        scores.append(accuracy)
        
    print("Accuracies for each fold:", scores)
        
    print("Mean accuracy:", sum(scores)/len(scores))

In [124]:
sklearn_knn("breast-cancer.data", 0, [1,2,3,4,5,7,8,9],7)

Accuracies for each fold: [0.6428571428571429, 0.8571428571428571, 0.7142857142857143, 0.75, 0.6785714285714286, 0.6785714285714286, 0.7857142857142857, 0.8214285714285714, 0.7857142857142857, 0.8571428571428571]
Mean accuracy: 0.7571428571428571


In [155]:
sklearn_knn("car.data",6,[0,1,2,3,4,5],9)

Accuracies for each fold: [0.6976744186046512, 0.813953488372093, 0.7441860465116279, 0.8255813953488372, 0.7383720930232558, 0.7790697674418605, 0.75, 0.7034883720930233, 0.7383720930232558, 0.7441860465116279]
Mean accuracy: 0.7534883720930233


In [93]:
sklearn_knn("hayes-roth.data",5,[],2)

Accuracies for each fold: [0.5384615384615384, 0.3076923076923077, 0.6153846153846154, 0.6923076923076923, 0.6153846153846154, 0.5384615384615384, 0.6923076923076923, 0.6153846153846154, 0.5384615384615384, 0.6153846153846154]
Mean accuracy: 0.5769230769230769


# Comparing My knn to sklearn knn using Hypothesis Testing using t-paired test

### Null Hypothesis: There is no significant difference in performance between my knn and sklearn knn
### Alternate Hypothesis: There is significant difference in  performance between both

In [156]:
from scipy import stats
def compare_knn(my_knn_metrics, sklearn_metrics):
    my_knn=np.array(my_knn_metrics)
    sklearn_knn=np.array(sklearn_metrics)

    t_statistic, p_value = stats.ttest_rel(my_knn, sklearn_knn)

    alpha = 0.05
    print(p_value)

    if p_value < alpha:
        print("Reject the null hypothesis: There is a significant difference in performance.")
    else:
        print("Failed to reject the null hypothesis: There is no significant difference in performance.")


## Comparing k-fold scores of both KNN's

### For Breast cancer dataset

In [159]:
compare_knn([0.6071428571428571, 0.75, 0.7142857142857143, 0.7857142857142857, 0.7142857142857143, 0.7857142857142857, 
             0.8214285714285714, 0.7857142857142857, 0.7857142857142857, 0.8214285714285714],
            [0.6428571428571429, 0.8571428571428571, 0.7142857142857143, 0.75, 0.6785714285714286, 0.6785714285714286, 
             0.7857142857142857, 0.8214285714285714, 0.7857142857142857, 0.8571428571428571])

0.9999999999999996
Failed to reject the null hypothesis: There is no significant difference in performance.


### For Hayes Roth dataset

In [161]:
compare_knn([0.6923076923076923, 0.6153846153846154, 0.6153846153846154, 0.7692307692307693, 0.7692307692307693, 0.8461538461538461, 
            0.6153846153846154, 0.8461538461538461, 0.7692307692307693, 0.46153846153846156],
            [0.5384615384615384, 0.3076923076923077, 0.6153846153846154, 0.6923076923076923, 0.6153846153846154, 
             0.5384615384615384, 0.6923076923076923, 0.6153846153846154, 0.5384615384615384, 0.6153846153846154])

0.036787497879786135
Reject the null hypothesis: There is a significant difference in performance.


### For Car dataset

In [160]:
compare_knn([0.877906976744186, 0.936046511627907, 0.8895348837209303, 0.9418604651162791, 0.9186046511627907, 0.9418604651162791, 
             0.9302325581395349, 0.9244186046511628, 0.8953488372093024, 0.9244186046511628],
            [0.6976744186046512, 0.813953488372093, 0.7441860465116279, 0.8255813953488372, 0.7383720930232558, 
             0.7790697674418605, 0.75, 0.7034883720930233, 0.7383720930232558, 0.7441860465116279])

4.3992895266255376e-08
Reject the null hypothesis: There is a significant difference in performance.
